# ECON 0150 | Replication Notebook

**Title:** Political Benefits and Unemployment

**Original Authors:** Ercoli, Perillo

**Original Date:** Fall 2024

---

This notebook replicates the analysis from a student final project in ECON 0150: Economic Data Analysis.

## About This Replication

**Research Question:** Is there a direct correlation between benefits given by a political party and unemployment rate?

**Data Source:** U.S. state-level unemployment rates and maximum weekly unemployment benefits (2021 and 2025)

**Methods:** OLS regression using first differences (change in unemployment vs change in benefits)

**Main Finding:** No significant relationship between change in benefits and change in unemployment (p = 0.291, R² = 0.02).

**Course Concepts Used:**
- First differences approach
- OLS regression
- Scatter plots with regression lines
- Comparing cross-sectional and panel approaches

---
## Step 0 | Setup

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

In [ ]:
# Load data from course website
base_url = 'https://tayweid.github.io/econ-0150/projects/replications/0014/data/'

data21 = pd.read_csv(base_url + 'data2021.csv')
data25 = pd.read_csv(base_url + 'data2025.csv')

print(f"2021 data: {len(data21)} states")
print(f"2025 data: {len(data25)} states")

---
## Step 1 | Data Preparation

In [ ]:
# Preview data
print("2021 columns:", data21.columns.tolist())
print("\n2025 columns:", data25.columns.tolist())

In [ ]:
# Prepare 2021 data
data21_clean = data21[['State', 'Unemployment Rate', 'Benefits', 'Party']].dropna().copy()
data21_clean = data21_clean.rename(columns={
    'Unemployment Rate': 'unrate21',
    'Benefits': 'benefits21'
})

# Prepare 2025 data
data25_clean = data25[['State', 'Unemployment Rate (%)', 'Max. Weekly Benefits']].dropna().copy()
# Clean benefits column - remove $ and convert to float
data25_clean['Max. Weekly Benefits'] = data25_clean['Max. Weekly Benefits'].str.replace('$', '', regex=False).str.replace(',', '').astype(float)
data25_clean = data25_clean.rename(columns={
    'Unemployment Rate (%)': 'unrate25',
    'Max. Weekly Benefits': 'benefits25'
})

print(f"2021 cleaned: {len(data21_clean)} states")
print(f"2025 cleaned: {len(data25_clean)} states")

In [ ]:
# Merge datasets
data = data21_clean.merge(data25_clean, on='State', how='inner')

# Calculate changes
data['unemployment_diff'] = data['unrate25'] - data['unrate21']
data['benefits_diff'] = data['benefits25'] - data['benefits21']

print(f"Merged data: {len(data)} states")
data.head()

---
## Step 2 | Data Exploration

In [ ]:
# Summary statistics
data[['unrate21', 'unrate25', 'benefits21', 'benefits25', 'unemployment_diff', 'benefits_diff']].describe()

In [ ]:
# Distribution of changes
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(data['unemployment_diff'], bins=15, edgecolor='black')
axes[0].axvline(0, color='red', linestyle='--')
axes[0].set_xlabel('Change in Unemployment Rate')
axes[0].set_title('Distribution of Unemployment Changes (2021-2025)')

axes[1].hist(data['benefits_diff'], bins=15, edgecolor='black')
axes[1].axvline(0, color='red', linestyle='--')
axes[1].set_xlabel('Change in Weekly Benefits ($)')
axes[1].set_title('Distribution of Benefits Changes (2021-2025)')

plt.tight_layout()
plt.show()

---
## Step 3 | Visualization

In [ ]:
# Cross-sectional: 2021
plt.figure(figsize=(10, 5))
sns.regplot(data=data, x='benefits21', y='unrate21', ci=None)
plt.xlabel('Maximum Weekly Benefits ($)')
plt.ylabel('Unemployment Rate (%)')
plt.title('Unemployment vs Benefits (2021)')
plt.show()

In [ ]:
# Cross-sectional: 2025
plt.figure(figsize=(10, 5))
sns.regplot(data=data, x='benefits25', y='unrate25', ci=None)
plt.xlabel('Maximum Weekly Benefits ($)')
plt.ylabel('Unemployment Rate (%)')
plt.title('Unemployment vs Benefits (2025)')
plt.show()

In [ ]:
# First differences: Changes
plt.figure(figsize=(10, 6))
sns.regplot(data=data, x='benefits_diff', y='unemployment_diff', ci=95)
plt.axhline(0, color='gray', linestyle='--', alpha=0.5)
plt.axvline(0, color='gray', linestyle='--', alpha=0.5)
plt.xlabel('Change in Maximum Weekly Benefits ($)')
plt.ylabel('Change in Unemployment Rate (%)')
plt.title('Change in Unemployment vs Change in Benefits (2021-2025)')
plt.grid(True)
plt.show()

---
## Step 4 | Statistical Analysis

In [ ]:
# OLS regression: First differences approach
model = smf.ols('unemployment_diff ~ benefits_diff', data=data).fit()
print(model.summary().tables[1])
print(f"\nR-squared: {model.rsquared:.3f}")

In [ ]:
# Compare with cross-sectional 2021
model_2021 = smf.ols('unrate21 ~ benefits21', data=data).fit()
print("2021 Cross-sectional:")
print(model_2021.summary().tables[1])
print(f"R-squared: {model_2021.rsquared:.3f}")

---
## Step 5 | Results Interpretation

### Key Findings

**First Differences Model:**
- **Intercept:** 0.44 (p = 0.005) - Average unemployment fell by 0.44 points
- **benefits_diff coefficient:** -0.0007 (p = 0.291) - NOT significant
- **R-squared:** 0.02 - Benefits changes explain almost none of the variation

### Interpretation

There is no statistically significant relationship between changes in unemployment benefits and changes in unemployment rates across U.S. states from 2021 to 2025.

This null finding suggests that:
1. Benefit levels may not be a major driver of unemployment rates
2. Other factors (economic conditions, industry composition) dominate
3. The sample size (50 states) may be too small to detect small effects

### Caveats

- Causality cannot be established with this design
- Omitted variables (e.g., state economic policies, COVID recovery) may confound
- Benefits affect job search behavior, but many other factors affect unemployment

---
## Replication Exercises

### Exercise 1: Party Effects
Does the relationship differ between Republican and Democratic states? Add party as a control variable.

### Exercise 2: Benefit Generosity
Calculate benefits as a percentage of median wage. Does this relative measure show different results?

### Exercise 3: Multiple Years
Gather more years of data (e.g., 2019, 2021, 2023, 2025) to create a panel dataset.

### Challenge Exercise
Research the economic literature on unemployment insurance and labor supply. What does theory predict about this relationship?

In [ ]:
# Your code for exercises
